# Scalar diffusion and convection-diffusion problem on channel geometry with uniform mesh using Gridap  

## Import libraries 

In [104]:
using Gridap

## Section 1:/ Diffusion Problem 
1. how to extend to non-linear diffusion? See [example](https://gridap.github.io/Tutorials/stable/pages/t004_p_laplacian/)
2. how to extend to time-dependent diffusion? See [example](https://gridap.github.io/Tutorials/dev/pages/t017_poisson_transient/). 
3. how to extend to vector-valued problems? See [example](https://gridap.github.io/Tutorials/stable/pages/t009_stokes/)
4. how to accomodate parallel computing? See [tutorial](https://gridap.github.io/Tutorials/dev/pages/t016_poisson_distributed/)

In [223]:
domain = (0,3,0,1)
partition = (30,10)
model = CartesianDiscreteModel(domain,partition)

CartesianDiscreteModel()

In [224]:
# Define Lagrangian reference element
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Vh = FESpace(model, reffe; conformity=:H1, dirichlet_tags = [7,8])
Uh = TrialFESpace(Vh, [0, 1]) # Left boundary: u = 0, right boundary: u = 1

TrialFESpace()

In [225]:
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

Measure()

In [238]:
# working version 
ff(x) = 0
a(u,v) = ∫( ∇(v)⋅∇(u) + u*u*v)*dΩ
b(v) = ∫( ff*v )*dΩ

b (generic function with 1 method)

In [239]:
op = AffineFEOperator(a,b,Uh,Vh)

AffineFEOperator()

In [240]:
ls = LUSolver()
solver = LinearFESolver(ls)

LinearFESolver()

In [244]:
?LUSolver

search: LUSolver



```
struct LUSolver <: LinearSolver end
```

Wrapper of the LU solver available in julia


In [249]:
print(op)

AffineFEOperator()

In [241]:
uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 300
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 10336619661980285595

In [243]:
?solve

search: solve solve! solver NLSolver LUSolver FESolver LinearFESolver



```
solve(nls::NonlinearSolver,op::NonlinearOperator)
```

Creates and uses a zero initial guess.

---

```
solve(ls::LinearSolver,A::AbstractMatrix,b::AbstractVector)
```

---

Solve that allocates, and sets initial guess to zero and returns the solution

---



In [242]:
writevtk(Ω,"results",cellfields=["uh"=>uh])

(["results.vtu"],)

## Section 2:/ Convection-Diffusion Problem 

In [215]:
domain = (0,3,0,1)
partition = (30,10)
model = CartesianDiscreteModel(domain,partition)

CartesianDiscreteModel()

In [216]:
# Define Lagrangian reference element
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Vh = FESpace(model, reffe; conformity=:H1, dirichlet_tags = [7,8])
Uh = TrialFESpace(Vh, [0, 1]) # Left boundary: u = 0, right boundary: u = 1

TrialFESpace()

In [217]:
degree = 1
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

Measure()

In [218]:
# working version 
ff(x) = 0
velocity(x) = VectorValue( 1.0, 0.0 )
a(u,v) = ∫( 0.1*∇(v)⋅∇(u) + (velocity ⋅ ∇(u))*v)*dΩ
b(v) = ∫( v*ff )*dΩ

b (generic function with 1 method)

In [219]:
op = AffineFEOperator(a,b,Uh,Vh)

AffineFEOperator()

In [220]:
ls = LUSolver()
solver = LinearFESolver(ls)

LinearFESolver()

In [221]:
uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 300
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 221304387296365017

In [222]:
writevtk(Ω,"results",cellfields=["uh"=>uh])

(["results.vtu"],)